import libraries

In [1]:
import tensorflow as tf
import numpy as np
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pylab as plt
from keras.preprocessing import image
from tensorflow.keras import utils
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import f1_score
import time

2023-05-06 11:30:02.740922: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-06 11:30:02.782723: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-06 11:30:02.783376: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-06 11:30:03.511855: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


functions

In [2]:
SAVED_MODEL_BEST_ACCURACY_RESULTS = []
SAVED_MODEL_BEST_LOSS_RESULTS = []
EPOCHS = 30

In [3]:
def transform(data):
    data = utils.load_img(data, color_mode='grayscale', target_size=(64, 64))
    data = utils.img_to_array(data)
    data = np.expand_dims(data, axis=0)
    data = data / 255.0
    return data

In [4]:
def get_examples_and_classes(dataset_dir, num):
    classes = sorted(os.listdir(dataset_dir))
    examples = []
    for i in range(len(classes)):
        class_dir = os.path.join(dataset_dir, classes[i])
        images = os.listdir(class_dir)
        examples.append(os.path.join(class_dir, images[num]))

    return classes, examples

In [5]:
def plot(accuracy, loss, val_accuracy, val_loss):
    fig, ax1 = plt.subplots()
    ax1.set_xlabel('epoch')

    ax1.set_ylabel('total loss')
    ax1.plot(loss,color='tab:red')
    ax1.plot( val_loss, color='tab:brown')
    ax1.tick_params(axis='y')

    ax2 = ax1.twinx()  
    ax2.set_ylabel('accuracy')  
    ax2.plot( accuracy, color='tab:blue')
    ax2.plot( val_accuracy, color='tab:green')
    ax2.tick_params(axis='y')
    fig.tight_layout()

In [6]:
def plot_many(title='title', data=[]):
    for i,o in enumerate(data):
        plt.figure(i)
        plt.plot([i for i in range(EPOCHS)],o, label=title)
        plt.legend(loc='upper left')

In [7]:
def max_or_min_avarage_val(list_of_data, accuracy=True):
    dict_all_l = {}
    for i, o in enumerate(list_of_data):
        dict_all_l[i] = sum(o)/len(o)
    if accuracy:
        return dict(sorted(dict_all_l.items(), key=lambda item: item[1], reverse=True))
    else:
        return dict(sorted(dict_all_l.items(), key=lambda item: item[1]))

In [8]:
def save_lists():
    t = time.ctime(time.time())[:-5]
    with open(f'accuracy {t}.txt', 'w') as f:
        for item in SAVED_MODEL_BEST_ACCURACY_RESULTS:
            f.write("%s\n" % item)
    # with open(f'loss {t}.txt', 'w') as f:
    #     for item in SAVED_MODEL_BEST_LOSS_RESULTS:
    #         f.write("%s\n" % item)

In [9]:
def read_list_from_file(filename):
    lst = []
    with open(filename, 'r') as file:
        for line in file.readlines():
            if line in ('', ' ','\n'):
                break
            line = line[1:-2].split(', ')
            lst.append([float(i.strip()) for i in line])
    return lst

In [10]:
def show_best_res(list_of_data):
    max_val = {}
    for i, o in enumerate(list_of_data):
        max_val[i] = max(o)
    return dict(sorted(max_val.items(), key=lambda item: item[1], reverse=True))

dataset

In [11]:
train_datagen = ImageDataGenerator(rescale = 1./255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

train_folder = "/home/jovyan/dataset/dataset2/train"
test_folder = "/home/jovyan/dataset/dataset2/test"

training_set = train_datagen.flow_from_directory(train_folder,
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'categorical',
                                                 color_mode='grayscale')

test_set = test_datagen.flow_from_directory(test_folder,
                                            target_size = (64, 64),
                                            batch_size = 16,
                                            class_mode = 'categorical',
                                            color_mode='grayscale')

Found 22734 images belonging to 164 classes.
Found 5682 images belonging to 164 classes.


model

In [20]:
def model(in_1=8, in_2=8, in_3=8, in_4=8):
    model = Sequential()
    model.add(Conv2D(filters=in_1,kernel_size=2, strides=(1,1), input_shape=(64,64,1), padding='valid', activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2,2)))
    model.add(BatchNormalization())
    

    model.add(Conv2D(filters=in_2,kernel_size=2, strides=1, padding = 'same', activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2,2)))
    
    model.add(BatchNormalization())

    model.add(Conv2D(filters=in_3,kernel_size=2, strides=1, padding ='same', activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2,2)))
    model.add(BatchNormalization())
    
    
    model.add(Conv2D(filters=in_4,kernel_size=2, strides=1, padding ='same', activation = 'relu'))
    
    model.add(BatchNormalization())

    model.add(Flatten())
    model.add(Dense(units = 512 , activation = 'tanh'))
    model.add(Dense(units = 164 , activation = 'softmax'))

    model.compile( optimizer = 'Adam', loss = 'categorical_crossentropy',metrics = ['accuracy'])

    return model
summary_example = model()
summary_example.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_32 (Conv2D)          (None, 63, 63, 8)         40        
                                                                 
 max_pooling2d_30 (MaxPoolin  (None, 31, 31, 8)        0         
 g2D)                                                            
                                                                 
 batch_normalization_32 (Bat  (None, 31, 31, 8)        32        
 chNormalization)                                                
                                                                 
 conv2d_33 (Conv2D)          (None, 31, 31, 8)         264       
                                                                 
 max_pooling2d_31 (MaxPoolin  (None, 15, 15, 8)        0         
 g2D)                                                            
                                                      

choose best hyperparameters

In [13]:
def model_and_best_hyperparameters():
    nums = [ 8,16, 32, 64]
    start_time1 = time.time()
    for i in nums:        
        start_time2 = time.time()
        
        ready_model = model(i,i*2,i*3, i*4)        
        trained_model = ready_model.fit(training_set, steps_per_epoch = 32, epochs = EPOCHS, validation_data = test_set)
        
        end_time2 = time.time()
        print("for one training time taken: {:.2f} seconds".format(end_time2 - start_time2))
        
        accuracy = trained_model.history['accuracy']
        loss = trained_model.history['loss']

        SAVED_MODEL_BEST_ACCURACY_RESULTS.append(accuracy)
        SAVED_MODEL_BEST_LOSS_RESULTS.append(loss)
        
    end_time1 = time.time()
    print("for all training time taken: {:.2f} seconds".format(end_time1 - start_time1))

In [14]:
model_and_best_hyperparameters()

Epoch 1/30


2023-05-06 11:30:06.174210: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


32/32 [==============================] - ETA: 0s - loss: 5.2450 - accuracy: 0.0146

2023-05-06 11:30:08.733071: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


32/32 [==============================] - 6s 146ms/step - loss: 5.2450 - accuracy: 0.0146 - val_loss: 5.1202 - val_accuracy: 0.0070
Epoch 2/30
32/32 [==============================] - 4s 141ms/step - loss: 5.0982 - accuracy: 0.0205 - val_loss: 5.2327 - val_accuracy: 0.0074
Epoch 3/30
32/32 [==============================] - 5s 148ms/step - loss: 4.9750 - accuracy: 0.0342 - val_loss: 5.3760 - val_accuracy: 0.0111
Epoch 4/30
32/32 [==============================] - 5s 152ms/step - loss: 4.8841 - accuracy: 0.0352 - val_loss: 5.4492 - val_accuracy: 0.0102
Epoch 5/30
32/32 [==============================] - 6s 179ms/step - loss: 4.8271 - accuracy: 0.0303 - val_loss: 5.4401 - val_accuracy: 0.0097
Epoch 6/30
32/32 [==============================] - 5s 156ms/step - loss: 4.7972 - accuracy: 0.0388 - val_loss: 5.5855 - val_accuracy: 0.0084
Epoch 7/30
32/32 [==============================] - 4s 130ms/step - loss: 4.7057 - accuracy: 0.0547 - val_loss: 5.5591 - val_accuracy: 0.0100
Epoch 8/30
32/32 

...

32/32 [==============================] - 10s 287ms/step - loss: 5.4059 - accuracy: 0.0127 - val_loss: 5.2586 - val_accuracy: 0.0106
Epoch 2/30
32/32 [==============================] - 9s 289ms/step - loss: 5.3144 - accuracy: 0.0186 - val_loss: 5.4899 - val_accuracy: 0.0106
Epoch 3/30
32/32 [==============================] - 9s 291ms/step - loss: 5.1386 - accuracy: 0.0244 - val_loss: 5.7522 - val_accuracy: 0.0049
Epoch 4/30
32/32 [==============================] - 9s 283ms/step - loss: 5.0681 - accuracy: 0.0273 - val_loss: 6.0871 - val_accuracy: 0.0044
Epoch 5/30
32/32 [==============================] - 9s 283ms/step - loss: 5.0287 - accuracy: 0.0332 - val_loss: 5.8383 - val_accuracy: 0.0088
Epoch 6/30
32/32 [==============================] - 9s 282ms/step - loss: 5.0128 - accuracy: 0.0264 - val_loss: 5.8395 - val_accuracy: 0.0062
Epoch 7/30
32/32 [==============================] - 9s 282ms/step - loss: 4.9379 - accuracy: 0.0361 - val_loss: 5.8533 - val_accuracy: 0.0088
Epoch 8/30
32/32

2023-05-06 11:40:41.489291: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


32/32 [==============================] - ETA: 0s - loss: 5.5751 - accuracy: 0.0088

2023-05-06 11:40:54.320478: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


32/32 [==============================] - 34s 1s/step - loss: 5.5751 - accuracy: 0.0088 - val_loss: 5.3447 - val_accuracy: 0.0060
Epoch 2/30
32/32 [==============================] - 43s 1s/step - loss: 5.3901 - accuracy: 0.0176 - val_loss: 5.6951 - val_accuracy: 0.0076
Epoch 3/30
32/32 [==============================] - 49s 2s/step - loss: 5.2770 - accuracy: 0.0176 - val_loss: 5.7805 - val_accuracy: 0.0072
Epoch 4/30
32/32 [==============================] - 50s 2s/step - loss: 5.2225 - accuracy: 0.0137 - val_loss: 5.9405 - val_accuracy: 0.0077
Epoch 5/30
32/32 [==============================] - 50s 2s/step - loss: 5.1630 - accuracy: 0.0303 - val_loss: 5.8553 - val_accuracy: 0.0074
Epoch 6/30
32/32 [==============================] - 52s 2s/step - loss: 5.1594 - accuracy: 0.0303 - val_loss: 6.0634 - val_accuracy: 0.0072
Epoch 7/30
32/32 [==============================] - 24s 741ms/step - loss: 5.1246 - accuracy: 0.0234 - val_loss: 6.1121 - val_accuracy: 0.0100
Epoch 8/30
32/32 [==========

In [16]:
print("sorted list of average  epochs' values for accuracy  ", max_or_min_avarage_val(SAVED_MODEL_BEST_ACCURACY_RESULTS))
print("sorted list of max  epochs' values for accuracy  ", show_best_res(SAVED_MODEL_BEST_ACCURACY_RESULTS))
save_lists()

sorted list of average  epochs' values for accuracy   {1: 0.11104551057020823, 2: 0.1001953125, 0: 0.09116854866345724, 3: 0.05363198419411977}
sorted list of max  epochs' values for accuracy   {2: 0.2216796875, 1: 0.189453125, 0: 0.1767578125, 3: 0.119140625}


In [ ]:
# file = 'accuracy Sat May  6 11:19:23.txt'
# l = read_list_from_file(file)
# print("sorted list of max  epochs' values for accuracy  ", show_best_res(l))

train and save model

In [ ]:
def final_model():
    ready_model = model(8,16,64)
    call_back = ModelCheckpoint(f"/home/jovyan/weights.hdf5", monitor='val_accuracy', verbose=0, save_best_only=True, save_weights_only=False, mode='auto')
    trained_model = model.fit(training_set, steps_per_epoch = 32, epochs = 30, validation_data = test_set, callbacks = [call_back])
    return trained_model


In [ ]:
# trained_model = final_model()

plot model

In [ ]:
# plot_many('accuracy', SAVED_MODEL_BEST_ACCURACY_RESULTS)


In [ ]:
if trained_model:
    plot(trained_model.history['accuracy'], trained_model.history['loss'], trained_model.history['val_accuracy'], trained_model.history['val_loss'])

predict

In [ ]:
predict_model = Sequential()
for layer in model.layers[:-1]:
    predict_model.add(layer)
predict_model.add(Dense(units=1, activation=None))
predict_model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
predict_model.summary()


predict_model.load_weights('weights.hdf5', skip_mismatch=True, by_name=True)

# # Load weights and biases from model manually
# for i in range(len(predict_model.layers) - 1):
#     predict_model.layers[i].set_weights(model.layers[i].get_weights())


In [ ]:
def predict(model, known_data_path, unknown_data_path):
    known_data = transform(known_data_path)
    unknown_data = transform(unknown_data_path)

    known_data = model.predict(known_data)
    unknown_data = model.predict(unknown_data)
    distance = unknown_data - known_data

    threshold = 0.018614814
    return  distance[0][0] > threshold, distance[0][0]

predict(predict_model, '/home/jovyan/11.jpg', '/home/jovyan/21.jpg')

plot prediction

In [ ]:
classes, examples1 = get_examples_and_classes(test_folder,1)
classes, examples2 = get_examples_and_classes(test_folder,2)
y_true = [i for i in range(len(classes))]
y_predict_tensors = []
y_predict_labels = []

for i in range(len(classes)):
    predict_label, predict_tensor = predict(predict_model, examples1[i], examples2[i])
    y_predict_tensors.append(predict_tensor)
    y_predict_labels.append(predict_label)    

In [ ]:
print(classification_report(y_true, y_predict_labels, target_names=classes))

choose best threshold

In [ ]:
thresholds = np.linspace(0.0001, 0.5, 1000)

# Calculate F1 score for each threshold
f1_scores = []
for threshold in thresholds:
    y_pred = (y_predict_tensors < threshold).astype(int)
    f1_scores.append(f1_score(y_true, y_pred, average='micro'))

# Choose the threshold that maximizes F1 score
best_threshold = thresholds[np.argmax(f1_scores)]
best_threshold